## **Importando pacotes**

In [92]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import sys


from sklearn.metrics 	     import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose 	     import ColumnTransformer
from sklearn.pipeline 	     import Pipeline
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing   import StandardScaler, PolynomialFeatures, OneHotEncoder, MinMaxScaler, PowerTransformer
from sklearn.impute          import KNNImputer

from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import SVC
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost                 import XGBClassifier
from lightgbm                import LGBMClassifier
from catboost                import CatBoostClassifier
from sklearn.dummy           import DummyClassifier
from sklearn.neighbors       import KNeighborsClassifier

from sklearn.feature_selection import SelectKBest
from category_encoders.target_encoder import TargetEncoder
from category_encoders.cat_boost import CatBoostEncoder


from feature_engine.creation import MathFeatures

sys.path.append('../utils')

from modelcrafterclass import ModelCrafter
import warnings
warnings.filterwarnings("ignore")

## **Carregando os dados**

In [93]:
data = pd.read_csv('../datasets_for_ml/dataset_for_train.csv')

In [94]:
data.head()

,Client,data_recente,target,qt_faltas,moda_staff_faltante,moda_servico_faltante,moda_servico_cancelado,moda_staff_cancelado,antecedencia,qt_cancelamentos,moda_staff_prestou_servico,moda_dia,qte_servicos_por_dia,media,mediana,desvio_padrao,min,max,qte_servico_recebido
0,KERT01,2018-06-20,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,JJ,Tuesday,1.5,84.666667,82.0,16.165808,70.0,102.0,3
1,COOM01,2018-06-15,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,SINEAD,Thursday,1.0,70.000000,70.0,NaN,70.0,70.0,1
2,PEDM01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,BECKY,Saturday,1.0,60.000000,60.0,NaN,60.0,60.0,1
3,BAIS01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0
4,FRAL01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,NaN,0,nenhum,nenhum,NaN,NaN,NaN,NaN,NaN,NaN,0


Apesar de haver valores faltantes, esses não são advindos de erros ou problemas. Na realidade, esses valores nulos tem seus significados dentro do contexto do nosso problema.

Por exemplo, um registro com a media faltante está associado a um novo cliente. Antecedência faltante indica que um cliente nunca fez um cancelamento e assim por diante.

Portanto, devemos achar uma maneira de introduzir essas informações na nossa base de dados

In [95]:
data.isna().sum().to_frame('qte. missing').sort_values(by='qte. missing',ascending=False)

,qte. missing
antecedencia,734
desvio_padrao,590
max,461
min,461
mediana,461
media,461
qte_servicos_por_dia,461
Client,0
moda_staff_prestou_servico,0
moda_dia,0


Como todos os valores nulos são numéricos, vamos substitui-los por -1

In [96]:
data = data.fillna(-1)

Vamos adicionar uma nova coluna, indicando se temos um novo cliente. Onde novo cliente é todo cliente sem nenhum histórico. O registro receberá 1 se for um cliente sem histórico e 0 se for um cliente com histórico.

In [97]:
data['novo_cliente'] = data.apply(lambda x: 1 if x['qte_servico_recebido'] == 0  and x['qt_cancelamentos'] == 0 and x['qt_faltas'] == 0 else 0,axis=1)

# **Breve EDA**

Ao todo há 798 registros, ou seja, 798 clientes únicos

In [98]:
data.shape

(798, 20)

Estamos lidando com um problema de classes desbalanceadas onde 751 (94%) clientes seguiram as políticas do salão enquanto 47 (5.9%) não seguiram

In [99]:
pd.concat([data['target'].value_counts().to_frame('Absoluto'),round(data['target'].value_counts(normalize=True).mul(100).to_frame('%'),2)],axis=1)

,Absoluto,%
target,,
0,751,94.11
1,47,5.89


Observamos que 442 (55%) são novos clientes e 356 (45%) são clientes com algum histórico.

In [100]:
pd.concat([data['novo_cliente'].value_counts().to_frame('Absoluto'),round(data['novo_cliente'].value_counts(normalize=True).mul(100).to_frame('%'),2)],axis=1)

,Absoluto,%
novo_cliente,,
1,442,55.39
0,356,44.61


Da observação acima, podemos notar um problema. Mais da metade do dataset são de novos clientes e, portanto, não tem nenhum histórico associado.

Clientes sem histórico podem ser um problema, pois a única informação que adquirimos sobre eles está no momento da reserva, o que pode não ser muito informativo.

Tomando somente os clientes que tem algum histórico também observamos um desbalanceamento dos dados, onde ~93% deles seguem a política do salão e 7% não seguem

In [101]:
data.query('novo_cliente == 0')['target'].value_counts(normalize=True).mul(100)

target
0    92.134831
1     7.865169
Name: proportion, dtype: float64

O mesmo ocorre para os clientes sem histórico

In [102]:
data.query('novo_cliente == 1')['target'].value_counts(normalize=True).mul(100)

target
0    95.701357
1     4.298643
Name: proportion, dtype: float64

Além disso, notamos que todas as colunas são exatamente iguais para todos os clientes sem histórico. Aquilo que não varia, enviesa. Portanto, acredito ser uma boa considerarmos apenas clientes com algum histórico.

In [103]:
data.query('novo_cliente == 1')

,Client,data_recente,target,qt_faltas,moda_staff_faltante,moda_servico_faltante,moda_servico_cancelado,moda_staff_cancelado,antecedencia,qt_cancelamentos,moda_staff_prestou_servico,moda_dia,qte_servicos_por_dia,media,mediana,desvio_padrao,min,max,qte_servico_recebido,novo_cliente
3,BAIS01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,-1.0,0,nenhum,nenhum,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1
4,FRAL01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,-1.0,0,nenhum,nenhum,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1
7,CHOT01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,-1.0,0,nenhum,nenhum,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1
8,KUZD01,2018-06-09,0,0,nenhum,nenhum,nenhum,nenhum,-1.0,0,nenhum,nenhum,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1
9,TINT01,2018-04-05,0,0,nenhum,nenhum,nenhum,nenhum,-1.0,0,nenhum,nenhum,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,MARR02,2018-06-09,1,0,nenhum,nenhum,nenhum,nenhum,-1.0,0,nenhum,nenhum,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1
793,CARS01,2018-05-25,1,0,nenhum,nenhum,nenhum,nenhum,-1.0,0,nenhum,nenhum,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1
794,SHMS01,2018-07-13,1,0,nenhum,nenhum,nenhum,nenhum,-1.0,0,nenhum,nenhum,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1
795,COLS01,2018-04-22,1,0,nenhum,nenhum,nenhum,nenhum,-1.0,0,nenhum,nenhum,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1


In [104]:
# selecionando somente clientes com histórico
data_to_train = data.query('novo_cliente == 0')

## **Definições**

Devemos estabelecer as métricas que vamos utilizar para a avaliação dos modelos. Vamos utilizar as três métricas citadas a seguir:

1. **Precision Score;**
2. **Recall Score;**
3. **F1 Score;**

A questão central do problema consiste em identificar faltas ou cancelamentos que fuja das políticas do salão. Tais ações acarretam em custos para o salão, desse modo, devemos selecionar se um cliente vai ou não seguir as políticas.

Vamos focar em maximizar a métrica f1-score, entretanto, no caso limite da f1 vamos focar em maximizar o recall.

Maximizando a f1 nós estamos maximizando precision e recall e maximizando o recall nós estamos interessados em identificar o máximo de clientes que não vão seguir com as políticas, entretanto, gostaríamos de fazer isso sem perder muito da precisão. 

# **Modelagem**

Definindo as features e o target

In [105]:
X = data_to_train.drop(['Client','data_recente','target'],axis=1)

y = data_to_train['target']

Definindo as variáveis numéricas e categóricas

In [106]:
categoricas = X.select_dtypes(include ='object').columns

numericas = X.select_dtypes(exclude ='object').columns

Instanciando o modelcrafter

In [107]:
modelcrafter = ModelCrafter(folds = 10)

Adicionando os modelos de classificação na estrutura

In [124]:
modelos = [('regressao_logistica', LogisticRegression(class_weight='balanced', max_iter=100000)),
           ('svc',SVC(class_weight='balanced',C=0.5)),
           ('knn',KNeighborsClassifier()),
           ('arvore',DecisionTreeClassifier(class_weight='balanced', random_state = 0 )),
           ('floresta',RandomForestClassifier(n_estimators=500,min_samples_split=10,class_weight='balanced',random_state = 0)),
           #('adaboost',AdaBoostClassifier(random_state = 0)),
           #('gradientboost',GradientBoostingClassifier(random_state = 0)),
           #('xgboost',XGBClassifier(random_state = 0)),
           #('lgbm',LGBMClassifier(class_weight='balanced',force_row_wise=True)),
           #('catboost',CatBoostClassifier(verbose=0)),
           ('dummy',DummyClassifier(strategy='uniform'))] 

modelcrafter.AddModel(modelos = modelos)

Estabelecendo uma pipeline inicial para os modelos

In [109]:
numerical_transform = Pipeline([("scaler",StandardScaler())])

categorical_transform = Pipeline([('encoder',TargetEncoder())])

preprocessor = ColumnTransformer([('categorical',categorical_transform, categoricas), 
                                  ('numeric', numerical_transform,numericas)],
                                remainder='passthrough')


model_pipeline = Pipeline([('preprocessor',preprocessor)])

model_pipeline.set_output(transform='pandas')

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   TargetEncoder())]),
                                                  Index(['moda_staff_faltante', 'moda_servico_faltante',
       'moda_servico_cancelado', 'moda_staff_cancelado',
       'moda_staff_prestou_servico', 'moda_dia'],
      dtype='object')),
                                                 ('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['qt_faltas', 'antecedencia', 'qt_cancelamentos', 'qte_servicos_por_dia',
       'media', 'mediana', 'desvio_padrao', 'min', 'max',
       'qte_servico_recebido', 'novo_cliente'],
      dtype='object'))]))])

Separando em treino e teste

In [110]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

Realizando uma baseline

In [111]:
baseline = modelcrafter.ValidacaoCruzada(X_train,y_train,model_pipeline)

baseline.sort_values(by='f1',ascending=False)

-----regressao_logistica-----
-----svc-----
-----knn-----
-----arvore-----
-----floresta-----
-----dummy-----


,precision,recall,f1
dummy,0.105185,0.691667,0.177719
regressao_logistica,0.107097,0.533333,0.173408
svc,0.098208,0.491667,0.157190
arvore,0.117063,0.250000,0.148737
knn,0.000000,0.000000,0.000000
floresta,0.000000,0.000000,0.000000


Notamos que alguns modelos tem todas as métricas iguais a zero.

A seguir as expressões para precision e recall:

$Precision = \dfrac{VP}{VP+FP}$

$Recall = \dfrac{VP}{VP+FN}$

A única maneira de obtermos precision e recall iguais a zero é obtendo os verdadeiros positivos iguais a zero. Além disso, nos warnings observamos que há uma possível divisão por zero no precision. Isso ocorre somente se os verdadeiros positivos e os falsos positivos forem iguais a zero, portanto, podemos entender que esses modelos estão estimando somente 0's.

Poderíamos alterar os hiperparâmetros desses modelos afim de conseguir obter resultados mais consistentes. Entretanto, por agora, podemos seguir com os modelos que não deram nenhum problema. 

In [122]:
numerical_transform = Pipeline([('polinomial',PolynomialFeatures(degree=3,interaction_only=True)),
                                ('power',PowerTransformer(standardize=False)),
                                ("scaler",StandardScaler())])

categorical_transform = Pipeline([('encoder',CatBoostEncoder())])

preprocessor = ColumnTransformer([('categorical',categorical_transform, categoricas), 
                                  ('numeric', numerical_transform,numericas)],
                                remainder='passthrough')


model_pipeline = Pipeline([('preprocessor',preprocessor)])

model_pipeline.set_output(transform='pandas')

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   CatBoostEncoder())]),
                                                  Index(['moda_staff_faltante', 'moda_servico_faltante',
       'moda_servico_cancelado', 'moda_staff_cancelado',
       'moda_staff_prestou_servico', 'moda_dia'],
      dtype='object')),
                                                 ('numeric',
                                                  Pipeline(steps=[('polinomial',
                                                                   PolynomialFeatures(degree=3,
                                                                                      interaction_only=True)),
                                                                  ('power',
                                                                   PowerTransformer(standardize=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['qt_faltas', 'antecedencia', 'qt_cancelamentos', 'qte_servicos_por_dia',
       'media', 'mediana', 'desvio_padrao', 'min', 'max',
       'qte_servico_recebido', 'novo_cliente'],
      dtype='object'))]))])

In [125]:
baseline = modelcrafter.ValidacaoCruzada(X_train,y_train,model_pipeline)

baseline.sort_values(by='f1',ascending=False)

-----regressao_logistica-----
-----svc-----
-----knn-----
-----arvore-----
-----floresta-----
-----dummy-----


,precision,recall,f1
svc,0.141443,0.516667,0.215372
regressao_logistica,0.086084,0.491667,0.141428
dummy,0.032168,0.183333,0.054551
arvore,0.008333,0.050000,0.014286
knn,0.000000,0.000000,0.000000
floresta,0.000000,0.000000,0.000000


In [114]:
model_pipeline.fit_transform(X_train,y_train)

AttributeError: This 'Pipeline' has no attribute 'fit_transform'